In [1]:
import numpy as np 
import pandas as pd 
import os
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from bayes_opt import BayesianOptimization

import pandas as pd
import numpy as np 
import tensorflow as tf
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier

from mlens.ensemble import SuperLearner
##### DON'T FORGET TO SET A MAX NUMBER OF ITERATIONS HERE. DON'T GO WITH -1 PLEASE
from sklearn.svm import SVC


/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
df = pd.read_csv("creditcard.csv")

df = df.drop(['Time','V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8'], axis =1)
df = df.dropna()


train_df = df.sample(frac=0.8, random_state=0)
test_df =  df.drop(train_df.index)

train_df = shuffle(train_df)
test_df = shuffle(test_df)

y_train = train_df.pop('Class').as_matrix()
y_test = test_df.pop('Class').as_matrix()
X_train = train_df.as_matrix()
X_test = test_df.as_matrix()



#X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)

In [3]:
# new block!
# Let's try to make some functions that will help us parse the config
from config import *
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# a function to create the pbounds dictionary for each model
def make_pbounds():
    
    # initialize an empty dictionary for all the pbounds of the models to go into
    pbounds_dict = {}
    
    # we should already have global variables for the config things
    # run through the models in model_config cause those are the ones we're actually using
    for model_name in model_config.keys():
        
        # initialize the model's pbounds dict
        model_pbounds = {}
        
        # check if the hparam's dictionary is empty. If so, we need to keep the pbounds dict empty
        ##### IF THE PBOUNDS DICT IS EMPTY, IT MEANS THE MODEL IS NOT TO BE OPTIMIZED, JUST TRAINED
        if model_config[model_name]['hyperparameters'] == {}:
            pbounds_dict[model_name] = model_pbounds
            continue
        
        # and go through all the hyperparameters in here
        for hparam_name in model_config[model_name]['hyperparameters'].keys():
            
            ##### WE WILL NEED SUPPORT TO ADD THE ALIAS HERE AND REMOVE IT AT THE COMPLETION OF OPTIMIZATION
            ##### WHEN WE ALLOW MULTIPLE MODELS FROM EACH TYPE
            
            if type(model_config[model_name]['hyperparameters'][hparam_name]) is not dict:
                continue
            
            # if 'range' is 'base', dig into the classification_models dict
            if model_config[model_name]['hyperparameters'][hparam_name]['range'] == 'base':
                
                model_pbounds[hparam_name] = classification_models[model_name]['hyperparameters'][hparam_name]['default_range']
            
            # else we're good to use this one
            else:
                model_pbounds[hparam_name] = model_config[model_name]['hyperparameters'][hparam_name]['range']
        
        # now we should have a completed model_pbounds, add it to pbounds_dict
        pbounds_dict[model_name] = model_pbounds
    
    # then return the pbounds_dict!
    return pbounds_dict

# function to import all the models
##### I have nooo idea if this will do it globally or if we'll need some help
def import_models():
    
    # loop through all the models in model_config and call their import statements written in classification_models
    for model_name in model_config.keys():
        import_statement = classification_models[model_name]['import_statement']
        
        exec(import_statement, globals())

# a function to determine which kind of error we'll be optimizing for
def error(preds, y_test):
    
    error_name = config['metric']
    
    if error_name == 'roc_auc':
        return roc_auc_score(preds, y_test)
    elif error_name == 'accuracy':
        return accuracy_score(preds, y_test)
    elif error_name == 'recall':
        return recall_score(preds, y_test)
    elif error_name == 'precision':
        return precision_score(preds, y_test)
    elif error_name == 'f1':
        return f1_score(preds, y_test)
    else:
        print('unsure what your metric is in the config so using accuracy instead')
        return accuracy_score(preds, y_test)

# we need a function that's going to create our whole fleet of base models
# we may need to change the config so that we have a reduced set of hyperparameters
# that we randomly create the models from
# ideally, we want these to be weak learners. Shallow models randomly set up
# we'll also be using the stacking classifier so we just need to put these into a list all good!
def create_base_model_set():
    
    from random import random
    import_models()
    
    # we need to make a list to store all the models!
    base_model_list = []
    
    
    for model_name in model_config.keys():
        
        # pull out the low and high number of models to create
        low_model_num = model_config[model_name]['num_in_base'][0]
        high_model_num = model_config[model_name]['num_in_base'][1]
        
        # loop through the (random) number of these random models we want to create
        for i in range(int((high_model_num - low_model_num)*random() + low_model_num)):
        
            # check if the hparam's dictionary is empty. If it is empty, we don't do any randomization!
            # Just go ahead and add the model to the list
            if model_config[model_name]['hyperparameters_for_base'] == {}:
                temp_str = 'base_model_list.append({}())'.format(model_name)
                exec(temp_str)
                continue

            # let's make the dictionary for this model's hparams
            hparam_dict = {}

            # and go through all the hyperparameters in here
            for hparam_name in model_config[model_name]['hyperparameters_for_base'].keys():
                
                # plainly set hparam that we're not changing
                if type(model_config[model_name]['hyperparameters_for_base'][hparam_name]) is not dict:
                    hparam_dict[hparam_name] = model_config[model_name]['hyperparameters_for_base'][hparam_name]
                    continue

                # if 'range' is 'base', dig into the classification_models dict
                if model_config[model_name]['hyperparameters_for_base'][hparam_name]['range'] == 'base':

                    hparam_range = classification_models[model_name]['hyperparameters'][hparam_name]['default_range']

                # else we're good to use this one
                else:
                    hparam_range = model_config[model_name]['hyperparameters_for_base'][hparam_name]['range']

                # next we need to select which values of these we're going to be using, randomly
                lower = hparam_range[0]
                upper = hparam_range[1]

                hparam_value = (upper - lower)*random() + lower

                # and now we need to check if it needs to be an int
                if classification_models[model_name]['hyperparameters'][hparam_name]['type'] == 'int':
                    hparam_value = int(hparam_value)

                hparam_dict[hparam_name] = hparam_value

            # then add the model with this hparam dict to the model_list
            temp_str = 'base_model_list.append({}(**hparam_dict))'.format(model_name)
            exec(temp_str)
    
    print('Base models:')
    print(base_model_list)
    print()
    
    return base_model_list

# now we have a list created of our base models!
# let's do basically the same process, but for the 'hidden' layers
# we'll just loop through the number of layers we need to create and BAM should have something
def create_hidden_model_layers():
    
    from random import random
    
    # this will be a list of (lists containing the models for one layer)
    hidden_list = []
    
    low_layer_num = config['num_hidden_layers'][0]
    high_layer_num = config['num_hidden_layers'][1]
    
    for layer_num in range(int((high_layer_num - low_layer_num)*random() + low_layer_num)):
        
        # we need to make a list to store all the models!
        layer_model_list = []


        for model_name in model_config.keys():

            # pull out the low and high number of models to create
            low_model_num = model_config[model_name]['num_in_mid'][0]
            high_model_num = model_config[model_name]['num_in_mid'][1]

            # loop through the (random) number of these random models we want to create
            for i in range(int((high_model_num - low_model_num)*random() + low_model_num)):

                # check if the hparam's dictionary is empty. If it is empty, we don't do any randomization!
                # Just go ahead and add the model to the list
                if model_config[model_name]['hyperparameters'] == {}:
                    temp_str = 'layer_model_list.append({}())'.format(model_name)
                    exec(temp_str)
                    continue

                # let's make the dictionary for this model's hparams
                hparam_dict = {}

                # and go through all the hyperparameters in here
                for hparam_name in model_config[model_name]['hyperparameters'].keys():
                    
                        
                    if type(model_config[model_name]['hyperparameters'][hparam_name]) is not dict:
                        hparam_dict[hparam_name] = model_config[model_name]['hyperparameters'][hparam_name]
                        continue
                    
                    
                    
                    # if 'range' is 'base', dig into the classification_models dict
                    if model_config[model_name]['hyperparameters'][hparam_name]['range'] == 'base':

                        hparam_range = classification_models[model_name]['hyperparameters'][hparam_name]['default_range']

                    # else we're good to use this one
                    else:
                        hparam_range = model_config[model_name]['hyperparameters'][hparam_name]['range']

                    # next we need to select which values of these we're going to be using, randomly
                    lower = hparam_range[0]
                    upper = hparam_range[1]

                    hparam_value = (upper - lower)*random() + lower

                    # and now we need to check if it needs to be an int
                    if classification_models[model_name]['hyperparameters'][hparam_name]['type'] == 'int':
                        hparam_value = int(hparam_value)

                    hparam_dict[hparam_name] = hparam_value
                

                # then add the model with this hparam dict to the model_list
                temp_str = 'layer_model_list.append({}(**hparam_dict))'.format(model_name)
                exec(temp_str)
        
        # then append this layer model list to the overall hidden list
        hidden_list.append(layer_model_list)
        
    print('Hidden models:')
    print(hidden_list)
    print()
    
    return hidden_list


# build the ensemble!
def build_ensemble(base_model_list, hidden_list):
    
    ens = SuperLearner(verbose=2) # lower this verbosity later if you don't want it
    ens.add(base_model_list)#, proba=True) # making probability on all these. We can talk about propagate_features later
    
    # then loop through all the other layers
    for layer in hidden_list:
        
        ens.add(layer)#, proba=True)
    
    return ens
    
    
# once we have all of these, we need to train everything except the meta-learner on this data.
# I hope that's possible to do that and just return the probabilities!
def train_layers(ens, X_train, y_train, X_test):
    
    print('Training base models')
    
    ### WE MAY NEED TO DO THIS ACROSS SEVERAL FOLDS!
    ens.fit(X_train, y_train)
    
    # we just want to return the prediction. This is hopefully one column for each model.
    # we want to train the metalearner on this set, with y_test, and have these train-test splitted.
    return ens.predict(X_test)


def error_function_meta(**hparams):
    
    
    model_name = model_name_temp_list[0] # I have no idea what I'm doing here
    
    # we need to go through each hparam in hparams and find out which ones need to be made into an int!
    for hparam_name in hparams.keys():

        if classification_models[model_name]['hyperparameters'][hparam_name]['type'] == 'int':
            hparams[hparam_name] = int(hparams[hparam_name])

    
    # then create the model
    # I surely hope we can do this in string format. May need to play around with global-ness
    # may also need some asterisks
    temp_list = []
    model_str = 'temp_list.append({}(**hparams))'.format(model_name)
    exec(model_str)
    model = temp_list[0]

    # fit the model
    model.fit(meta_x_train, meta_y_train)
    
    preds = model.predict(meta_x_test)
    
    
    try:
        temp_error = error(preds, meta_y_test)
    except:
        temp_error = 0

    return temp_error

# what the heck am I doing
model_name_temp_list = []

# then the meta learner
def run_meta_optimization(ens):
    
    # then make the pbounds dictionary
    pbounds_dict = make_pbounds()
    
    # now we need to get a list of models that we actually want to optimize
    meta_model_name_list = []
    for meta_model_name in model_config.keys():
        
        if model_config[meta_model_name]['metalearner']:
            
            meta_model_name_list.append(meta_model_name)
    
    # and then loop through each one of these models we'll be trying!
    max_target = 0
    max_meta_dict = {}
    for meta_model_name in meta_model_name_list:
        
        print('Optimizing meta learner {}'.format(meta_model_name))
        
        
        ##### we should add some time element in here too, that would be cool.

        # get the pbounds
        model_pbounds = pbounds_dict[meta_model_name]
        
        # ugh we need to set the name and everything
        #exec('model_name = {}'.format(meta_model_name), globals())
        if(len(model_name_temp_list) == 0):
            model_name_temp_list.append(meta_model_name)
        else:
            model_name_temp_list[0] = meta_model_name

        optimizer = BayesianOptimization(
            f=error_function_meta,
            pbounds=model_pbounds)
        optimizer.maximize(init_points=2, n_iter=config['meta_learner_its'], xi=0.5)
        max_hparams = optimizer.max
        
        target = max_hparams['target']
        
        print()
        print(' Finished {} optimization. Target: {}'.format(meta_model_name, target))
        
        if target > max_target:
            
            print('  New max target found!')
            
            # reassign the target
            max_target = target
            
            # then update the max_meta_dict
            max_meta_dict = {}
            max_meta_dict[meta_model_name] = max_hparams
    
    
    max_meta_dict['layers'] = ens
    
    print()
    print('Finished entire optimization')
    print('Max model found, with target {}:'.format(max_target))
    print(max_meta_dict)    
    
def run():
    
    run_meta_optimization(hparam_max_dict)
    
    
### OUT HERE WE'RE GOING TO HAVE TO SPLIT UP THE TRAINING AND TEST DATA FOR THE META LEARNER

/Users/michael/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
from config import *
import warnings
warnings.filterwarnings('ignore')

base_model_list = create_base_model_set()
hidden_list = create_hidden_model_layers()
ens = build_ensemble(base_model_list, hidden_list)
ens_preds = train_layers(ens, X_train, y_train, X_test)
print(ens_preds.shape)
meta_x_train, meta_x_test, meta_y_train, meta_y_test = train_test_split(ens_preds, y_test, train_size=0.75, test_size=0.25)
print(meta_x_train.shape)
run_meta_optimization(ens)

Base models:
[LogisticRegression(C=7.818638382609076, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.005334167521682301,
                   verbose=0, warm_start=False), LogisticRegression(C=1.5108631658759453, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.007804297640067662,
                   verbose=0, warm_start=False), SVC(C=4.343216723674997, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=1000, probability=True, random_state=None,
    shrinking=True, tol=0.00010487383333886062, verbose=

Processing layer-1             done | 00:02:58
Processing layer-2             

KeyboardInterrupt: 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn
model2 = knn(n_neighbors = 1)
model4 = knn(n_neighbors = 2)
model8 = knn(n_neighbors = 3)
model16 = knn(n_neighbors = 4)
model32 = knn(n_neighbors = 5)
model64 = knn(n_neighbors = 6)
model128 = knn(n_neighbors = 7)
model256 = knn(n_neighbors = 8)

model_list = [model2, model4, model8, model16, model32, model64, model128, model256]
ens_list = list(zip(['1','2','3','4','5','6','7','8'], model_list))

i = 0
for model in model_list:
    
    i += 1
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #acc_err = accuracy_score(preds, y_test)
    roc_err = roc_auc_score(preds, y_test)
    recall_err = recall_score(preds, y_test)
    prec_err = precision_score(preds, y_test)
    #f1_err = f1_score(preds, y_test)
    
    print(2**i)
    #print(' accuracy: {}'.format(acc_err))
    print(' roc: {}'.format(roc_err))
    print(' recall: {}'.format(recall_err))
    print(' precision: {}'.format(prec_err))
    #print(' f1: {}'.format(f1_err))
    print()

# train the stacker
stack_class = StackingClassifier(ens_list)
stack_class.fit(X_train, y_train)
preds = stack_class.predict(X_test)
#acc_err = accuracy_score(preds, y_test)
roc_err = roc_auc_score(preds, y_test)
recall_err = recall_score(preds, y_test)
prec_err = precision_score(preds, y_test)
#f1_err = f1_score(preds, y_test)
print('Stacked ensemble:')
#print(' accuracy: {}'.format(acc_err))
print(' roc: {}'.format(roc_err))
print(' recall: {}'.format(recall_err))
print(' precision: {}'.format(prec_err))
#print(' f1: {}'.format(f1_err))

In [ ]:
sh_list = ['soft', 'hard']
model_list_slices = [[0,1], [0,2], [0,3], [1,2], [1,3], [2,3], [0,1,2], [0,1,3], [0,2,3], [1,2,3], [0,1,2,3]]

In [ ]:
def error_function(**kwargs):#lr, n_est, md, msf, c, ada_n_est, nn, model_list_index, sh_index):
    
    print(kwargs)
    sh_index = int(sh_index)
    model_list_index = int(model_list_index)
    msf = int(msf)
    md = int(md)
    n_est = int(n_est)
    ada_n_est = int(ada_n_est)
    nn = int(nn)
    
    sh_choice = sh_list[sh_index]
    model_slice = model_list_slices[model_list_index]
    
    # create an error function based on the inputs
    ##### DON'T FORGET A LOT OF THESE CAN USE -1 CORES
    xgb_model = xgb(learning_rate=lr, n_estimators=n_est, max_depth=md, min_samples_leaf=msf)
    lr_model = LogisticRegression(C=c, solver='lbfgs')
    knn_model = KNeighborsClassifier(n_neighbors=nn)
    ada = AdaBoostClassifier(n_estimators=ada_n_est)
    
    models = [('1', xgb_model), ('2', lr_model), ('3', knn_model), ('4', ada)]
    
    model_choice = [models[n] for n in model_slice]
    
    
    
    total_error = 0
    
    for i in range(1):
        #X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)
        
        ens = VotingClassifier(model_choice, voting=sh_choice)
        ens.fit(X_train, y_train)
        preds = ens.predict(X_test)
        
        try:
            error = roc_auc_score(preds, y_test)
        except:
            error = 0
        
        total_error += error
    
    error = total_error
    
    return error

In [ ]:
pbounds = {'lr': (0.00001, 1), 
           'n_est': (1, 10), 
           'md': (1,100), 
           'msf': (1, 10),
           'c': (0.01, 10),
           'ada_n_est': (1, 100),
           'nn': (1, 100),
           'sh_index': (0, 1),
           'model_list_index': (0, len(model_list_slices) - 1)}

optimizer = BayesianOptimization(
    f=error_function,
    pbounds=pbounds
)

In [ ]:
optimizer.maximize(init_points=2, n_iter=1, xi=0.5)

In [ ]:
print(optimizer.max)